<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="cognitiveclass.ai logo">
</center>


# Machine Learning Foundation

## Course 4, Part c: Clustering Methods LAB


# Clustering Methods Exercises


## Introduction

This lab uses a dataset on wine quality. The data set contains various chemical properties of wine, such as acidity, sugar, pH, and alcohol. It also contains a quality metric (3-9, with highest being better) and a color (red or white). The name of the file is `Wine_Quality_Data.csv`.

We will be using the chemical properties (i.e. everything but quality and color) to cluster the wine. Though this is unsupervised learning, there are interesting semi-supervised extensions relating clustering results onto color and quality.


In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import seaborn as sns, pandas as pd, numpy as np

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns


## Question 1

*   Import the data and examine the features.
*   Note which are continuous, categorical, and boolean.
*   How many entries are there for the two colors and range of qualities?
*   Make a histogram plot of the quality for each of the wine colors.


In [ ]:
### BEGIN SOLUTION
# BÀI TẬP 1: Khám phá dữ liệu về chất lượng rượu

# Import dữ liệu từ CSV về chất lượng rượu vang
# Dataset chứa các thuộc tính hóa học (acidity, pH, alcohol, ...) và chất lượng (3-9)
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%202/Wine_Quality_Data.csv")

# Hiển thị 4 dòng đầu tiên với transpose để dễ quan sát
data.head(4).T

In [ ]:
data.shape

The data types for each entry. The implementation of K-means in Scikit-learn is designed only to work with continuous data (even though it is sometimes used with categorical or boolean types). Fortunately, all the columns we will be using (everything except quality and color) are continuous.


In [ ]:
data.dtypes

The number of entries for each wine color.


In [ ]:
data.color.value_counts()

The distribution of quality values.


In [ ]:
data.quality.value_counts().sort_index()

Now for the histogram.


In [ ]:
# seaborn styles
sns.set_context('notebook')
sns.set_style('white')

# custom colors
red = sns.color_palette()[2]
white = sns.color_palette()[4]

# set bins for histogram
bin_range = np.array([3, 4, 5, 6, 7, 8, 9])

# plot histogram of quality counts for red and white wines
ax = plt.axes()
for color, plot_color in zip(['red', 'white'], [red, white]):
    q_data = data.loc[data.color==color, 'quality']
    q_data.hist(bins=bin_range, 
                alpha=0.5, ax=ax, 
                color=plot_color, label=color)
    

ax.legend()
ax.set(xlabel='Quality', ylabel='Occurrence')

# force tick labels to be in middle of region
ax.set_xlim(3,10)
ax.set_xticks(bin_range+0.5)
ax.set_xticklabels(bin_range);
ax.grid('off')
### END SOLUTION

## Question 2

*   Examine the correlation and skew of the relevant variables--everything except color and quality (without dropping these columns from our data).
*   Perform any appropriate feature transformations and/or scaling.
*   Examine the pairwise distribution of the variables with pairplots to verify scaling and normalization efforts.


In [ ]:
### BEGIN SOLUTION
float_columns = [x for x in data.columns if x not in ['color', 'quality']]

# The correlation matrix
corr_mat = data[float_columns].corr()

# Strip out the diagonal values for the next step
for x in range(len(float_columns)):
    corr_mat.iloc[x,x] = 0.0
    
corr_mat

In [ ]:
# Pairwise maximal correlations
corr_mat.abs().idxmax()

And an examination of the skew values in anticipation of transformations.


In [ ]:
skew_columns = (data[float_columns]
                .skew()
                .sort_values(ascending=False))

skew_columns = skew_columns.loc[skew_columns > 0.75]
skew_columns

In [ ]:
# Thực hiện biến đổi logarit cho các cột có độ lệch cao (skewed)
# Log transformation giúp giảm độ lệch và làm phân phối gần với phân phối chuẩn hơn
# np.log1p = log(1 + x) để tránh log(0)
for col in skew_columns.index.tolist():
    data[col] = np.log1p(data[col])


Perform feature scaling.


In [ ]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa dữ liệu (Feature Scaling)
# StandardScaler chuyển đổi dữ liệu về mean=0 và std=1
# Điều này quan trọng cho K-means vì thuật toán dựa trên khoảng cách
sc = StandardScaler()
data[float_columns] = sc.fit_transform(data[float_columns])

data.head(4)

Finally, the pairplot of the transformed and scaled features.


In [ ]:
sns.set_context('notebook')
sns.pairplot(data[float_columns + ['color']], 
             hue='color', 
             hue_order=['white', 'red'],
             palette={'red':red, 'white':'gray'});
### END SOLUTION

## Question 3

*   Fit a K-means clustering model with two clusters.
*   Examine the clusters by counting the number of red and white wines in each cluster.


In [ ]:
from sklearn.cluster import KMeans
### BEGIN SOLUTION
# BÀI TẬP 3: Áp dụng K-means clustering

# Khởi tạo mô hình K-means với 2 clusters
# n_clusters=2: phân thành 2 nhóm (có thể tương ứng với rượu đỏ và trắng)
# random_state=42: để kết quả có thể tái tạo
km = KMeans(n_clusters=2, random_state=42)

# Huấn luyện mô hình trên các thuộc tính hóa học
km = km.fit(data[float_columns])

# Dự đoán cluster cho mỗi điểm dữ liệu và lưu vào cột 'kmeans'
data['kmeans'] = km.predict(data[float_columns])

In [ ]:
(data[['color','kmeans']]
 .groupby(['kmeans','color'])
 .size()
 .to_frame()
 .rename(columns={0:'number'}))
### END SOLUTION

## Question 4

*   Now fit K-Means models with cluster values ranging from 1 to 20.
*   For each model, store the number of clusters and the inertia value.
*   Plot cluster number vs inertia. Does there appear to be an ideal cluster number?


In [ ]:
### BEGIN SOLUTION
# BÀI TẬP 4: Tìm số lượng cluster tối ưu bằng Elbow Method

# Tạo danh sách để lưu kết quả của các mô hình
km_list = list()

# Thử với số cluster từ 1 đến 20
for clust in range(1,21):
    # Khởi tạo và huấn luyện K-means với số cluster = clust
    km = KMeans(n_clusters=clust, random_state=42)
    km = km.fit(data[float_columns])
    
    # Lưu số cluster, inertia (tổng khoảng cách bình phương đến centroid), và model
    # Inertia thấp hơn = clustering tốt hơn, nhưng cần cân nhắc với số cluster
    km_list.append(pd.Series({'clusters': clust, 
                              'inertia': km.inertia_,
                              'model': km}))

In [ ]:
plot_data = (pd.concat(km_list, axis=1)
             .T
             [['clusters','inertia']]
             .set_index('clusters'))

ax = plot_data.plot(marker='o',ls='-')
ax.set_xticks(range(0,21,2))
ax.set_xlim(0,21)
ax.set(xlabel='Cluster', ylabel='Inertia');
### END SOLUTION

## Question 5

*   Fit an agglomerative clustering model with two clusters.
*   Compare the results to those obtained by K-means with regards to wine color by reporting the number of red and white observations in each cluster for both K-means and agglomerative clustering.
*   Visualize the dendrogram produced by agglomerative clustering. *Hint:* SciPy has a module called [`cluster.hierarchy`](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2022-01-01#module-scipy.cluster.hierarchy) that contains the `linkage` and `dendrogram` functions required to create the linkage map and plot the resulting dendrogram.


In [ ]:
from sklearn.cluster import AgglomerativeClustering
### BEGIN SOLUTION
# BÀI TẬP 5: Áp dụng Agglomerative Clustering (phân cụm phân cấp)

# Khởi tạo Agglomerative Clustering với 2 clusters
# n_clusters=2: số lượng cluster cuối cùng
# linkage='ward': phương pháp liên kết minimizing variance trong cluster
# compute_full_tree=True: tính toán cây phân cấp đầy đủ để vẽ dendrogram
ag = AgglomerativeClustering(n_clusters=2, linkage='ward', compute_full_tree=True)
ag = ag.fit(data[float_columns])

# Dự đoán cluster và lưu vào cột 'agglom'
data['agglom'] = ag.fit_predict(data[float_columns])

Note that cluster assignment is arbitrary, the respective primary cluster numbers for red and white may not be identical to the ones below and also may not be the same for both K-means and agglomerative clustering.


In [ ]:
# First, for Agglomerative Clustering:
(data[['color','agglom','kmeans']]
 .groupby(['color','agglom'])
 .size()
 .to_frame()
 .rename(columns={0:'number'}))

In [ ]:
# Comparing with KMeans results:
(data[['color','agglom','kmeans']]
 .groupby(['color','kmeans'])
 .size()
 .to_frame()
 .rename(columns={0:'number'}))

In [ ]:
# Comparing results:
(data[['color','agglom','kmeans']]
 .groupby(['color','agglom','kmeans'])
 .size()
 .to_frame()
 .rename(columns={0:'number'}))

Though the cluster numbers are not identical, the clusters are very consistent within a single wine variety (red or white).

And here is a plot of the dendrogram created from agglomerative clustering.


In [ ]:
# Import module hierarchy từ SciPy để vẽ dendrogram (sơ đồ cây phân cấp)
from scipy.cluster import hierarchy

# Tạo ma trận linkage từ cây phân cụm đã tính
# Z chứa thông tin về cách các cluster được gộp lại từng bước
Z = hierarchy.linkage(ag.children_, method='ward')

# Tạo figure với kích thước lớn để dễ quan sát
fig, ax = plt.subplots(figsize=(15,5))

# Vẽ dendrogram (sơ đồ cây phân cấp)
# orientation='top': cây phát triển từ dưới lên
# p=30: chỉ hiển thị 30 nhánh cuối cùng (để đơn giản hóa)
# truncate_mode='lastp': chỉ hiển thị p nhánh cuối
# show_leaf_counts=True: hiển thị số lượng điểm dữ liệu trong mỗi nhánh
den = hierarchy.dendrogram(Z, orientation='top', 
                           p=30, truncate_mode='lastp',
                           show_leaf_counts=True, ax=ax)
### END SOLUTION

## Question 6

In this question, we are going to explore clustering as a form of feature engineering.

*   Create a **binary** target variable `y`, denoting if the quality is greater than 7 or not.
*   Create a variable called `X_with_kmeans` from `data`, by dropping the columns "quality", "color" and "agglom" from the dataset. Create `X_without_kmeans` from that by dropping "kmeans".
*   For both datasets, using **[StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2022-01-01)** with 10 splits, fit 10 Random Forest Classifiers and find the mean of the ROC-AUC scores from these 10 classifiers.
*   Compare the average roc-auc scores for both models, the one using the KMeans cluster as a feature and the one that doesn't use it.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit

### BEGIN SOLUTION
# BÀI TẬP 6: Sử dụng Clustering làm Feature Engineering

# Tạo biến target nhị phân: 1 nếu chất lượng > 7, 0 nếu ngược lại
y = (data['quality'] > 7).astype(int)

# Tạo feature set có KMeans cluster
X_with_kmeans = data.drop(['agglom', 'color', 'quality'], axis=1)

# Tạo feature set không có KMeans cluster
X_without_kmeans = X_with_kmeans.drop('kmeans', axis=1)

# Sử dụng StratifiedShuffleSplit để chia dữ liệu thành 10 splits
# Stratified đảm bảo tỷ lệ class được giữ nguyên trong mỗi split
sss = StratifiedShuffleSplit(n_splits=10, random_state=6532)

def get_avg_roc_10splits(estimator, X, y):
    """
    Hàm tính ROC-AUC trung bình qua 10 splits
    ROC-AUC đo lường khả năng phân loại của mô hình (0.5-1.0, càng cao càng tốt)
    """
    roc_auc_list = []
    # Lặp qua 10 splits
    for train_index, test_index in sss.split(X, y):
        # Chia train/test set
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Huấn luyện mô hình
        estimator.fit(X_train, y_train)
        
        # Dự đoán nhãn và xác suất
        y_predicted = estimator.predict(X_test)
        y_scored = estimator.predict_proba(X_test)[:, 1]
        
        # Tính ROC-AUC score
        roc_auc_list.append(roc_auc_score(y_test, y_scored))
    
    # Trả về ROC-AUC trung bình
    return np.mean(roc_auc_list)

# Khởi tạo Random Forest Classifier
estimator = RandomForestClassifier()

# So sánh hiệu suất với và không có KMeans cluster feature
roc_with_kmeans = get_avg_roc_10splits(estimator, X_with_kmeans, y)
roc_without_kmeans = get_avg_roc_10splits(estimator, X_without_kmeans, y)

print("Without kmeans cluster as input to Random Forest, roc-auc is \"{0}\"".format(roc_without_kmeans))
print("Using kmeans cluster as input to Random Forest, roc-auc is \"{0}\"".format(roc_with_kmeans))

Let's now explore if the number of clusters have an effect in this improvement.

*   Create the basis training set from `data` by restricting to float_columns.
*   For $n = 1, \ldots, 20$, fit a KMeans algorithim with $n$ clusters. **[One-hot encode]()** it and add it to the **basis** training set. Don't add it to the previous iteration.
*   Fit 10 **Logistic Regression** models and compute the average roc-auc-score.
*   Plot the average roc-auc scores.


In [ ]:
from sklearn.linear_model import LogisticRegression

# Tạo tập dữ liệu cơ sở chỉ với các cột số thực
X_basis = data[float_columns]
sss = StratifiedShuffleSplit(n_splits=10, random_state=6532)

def create_kmeans_columns(n):
    """
    Hàm tạo features từ KMeans clustering với n clusters
    One-hot encode các cluster labels để tạo features nhị phân
    """
    # Fit KMeans với n clusters
    km = KMeans(n_clusters=n)
    km.fit(X_basis)
    
    # Dự đoán cluster cho mỗi điểm
    km_col = pd.Series(km.predict(X_basis))
    
    # One-hot encode cluster labels
    # Mỗi cluster trở thành một cột binary (0 hoặc 1)
    km_cols = pd.get_dummies(km_col, prefix='kmeans_cluster')
    
    # Kết hợp features gốc với cluster features
    return pd.concat([X_basis, km_cols], axis=1)

# Sử dụng Logistic Regression thay vì Random Forest
estimator = LogisticRegression()

# Thử với số cluster từ 1 đến 20
ns = range(1, 21)

# Tính ROC-AUC trung bình cho mỗi số cluster
roc_auc_list = [get_avg_roc_10splits(estimator, create_kmeans_columns(n), y)
                for n in ns]

# Vẽ biểu đồ kết quả
ax = plt.axes()
ax.plot(ns, roc_auc_list)
ax.set(
    xticklabels= ns,
    xlabel='Number of clusters as features',
    ylabel='Average ROC-AUC over 10 iterations',
    title='KMeans + LogisticRegression'
)
ax.grid(True)
### END SOLUTION

***

### Machine Learning Foundation (C) 2020 IBM Corporation
